# Tree Methods Consulting Project 

Te han contratado en una empresa de alimentos para perros para tratar de predecir por qué algunos lotes de su comida para perros se están echando a perder mucho más rápido de lo esperado. Desafortunadamente, esta empresa no ha actualizado su maquinaria, lo que significa que las cantidades de los cinco químicos conservantes que están utilizando pueden variar mucho. Pero, ¿cuál es el químico que tiene el efecto más fuerte?

La empresa de alimentos para perros mezcla un lote de conservantes que contiene 4 químicos conservantes diferentes (A, B, C, D) y luego lo completa con un químico "de relleno". Los científicos alimentarios creen que uno de los conservantes A, B, C o D es el que está causando el problema, ¡pero necesitan que tú averigües cuál es!

Usa Machine Learning con Random Forest (Bosques Aleatorios) para descubrir qué parámetro tiene el mayor poder predictivo, identificando así qué químico causa el deterioro temprano. Luego, descubre cómo puedes decidir cuál químico es el problema.

Pres_A: Porcentaje del conservante A en la mezcla
Pres_B: Porcentaje del conservante B en la mezcla
Pres_C: Porcentaje del conservante C en la mezcla
Pres_D: Porcentaje del conservante D en la mezcla
Spoiled: Etiqueta que indica si el lote de comida para perros se echó a perder o no
Piensa cuidadosamente sobre lo que este problema te está pidiendo resolver. Aunque utilizaremos Machine Learning para resolverlo, no será con el típico flujo de trabajo de dividir los datos en conjuntos de entrenamiento y prueba. Si esto te confunde, salta a la sección de solución con código para seguir el paso a paso.



# Good Luck!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('footPet').getOrCreate()

25/01/08 07:01:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
data = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [8]:
data.head(2)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0)]

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [11]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
final_data = output.select('features', 'Spoiled')

In [15]:
final_data.show()

+-------------------+-------+
|           features|Spoiled|
+-------------------+-------+
| [4.0,2.0,12.0,3.0]|    1.0|
| [5.0,6.0,12.0,7.0]|    1.0|
| [6.0,2.0,13.0,6.0]|    1.0|
| [4.0,2.0,12.0,1.0]|    1.0|
| [4.0,2.0,12.0,3.0]|    1.0|
|[10.0,3.0,13.0,9.0]|    1.0|
| [8.0,5.0,14.0,5.0]|    1.0|
| [5.0,8.0,12.0,8.0]|    1.0|
| [6.0,5.0,12.0,9.0]|    1.0|
| [3.0,3.0,12.0,1.0]|    1.0|
| [9.0,8.0,11.0,3.0]|    1.0|
|[1.0,10.0,12.0,3.0]|    1.0|
|[1.0,5.0,13.0,10.0]|    1.0|
|[2.0,10.0,12.0,6.0]|    1.0|
|[1.0,10.0,11.0,4.0]|    1.0|
| [5.0,3.0,12.0,2.0]|    1.0|
| [4.0,9.0,11.0,8.0]|    1.0|
| [5.0,1.0,11.0,1.0]|    1.0|
|[4.0,9.0,12.0,10.0]|    1.0|
| [5.0,8.0,10.0,9.0]|    1.0|
+-------------------+-------+
only showing top 20 rows



In [14]:
#train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
from pyspark.ml.classification import RandomForestClassifier

In [20]:
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')

In [25]:
rfc_model = rfc.fit(final_data)

In [28]:
final_data.head(1)

[Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [29]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0184, 1: 0.0191, 2: 0.9408, 3: 0.0217})